In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
# Path to the data txt file on disk.
data_path = "hindi.txt"
# open the file eng-spa.txt and read
lines= pd.read_table(data_path,  names =['source', 'target', 'comments'])
#printing sample data from lines
lines.sample(6)

,source,target,comments
52,"Any breast lumps, pain or soreness?","कोई स्तन गांठ, दर्द या कष्ट?",CC-BY 2.0 (France) Attribution: tatoeba.org #1...
90,Any lesions or sores?,कोई घाव?,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
29,Neck,गरदन,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
35,Any pain when you take a deep breath?,कोई भी दर्द जब आप एक गहरी सांस लेते हैं?,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
38,Have you been exposed to anyone with tuberculo...,क्या आप तपेदिक के साथ किसी के संपर्क में हैं?,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
59,Any heartburn?,कोई अम्लशूल?,CC-BY 2.0 (France) Attribution: tatoeba.org #3...


In [4]:
# convert source and target text to Lowercase 
lines.source=lines.source.apply(lambda x: x.lower())
lines.target=lines.target.apply(lambda x: x.lower())
# Remove quotes from source and target text
lines.source=lines.source.apply(lambda x: re.sub("'", '', x))
lines.target=lines.target.apply(lambda x: re.sub("'", '', x))
# create a set of all special characters
special_characters= set(string.punctuation)
# Remove all the special characters
lines.source = lines.source.apply(lambda x: ''.join(char1 for char1 in x if char1 not in special_characters))
lines.target = lines.target.apply(lambda x: ''.join(char1 for char1 in x if char1 not in special_characters))
# Remove digits from source and target sentences
num_digits= str.maketrans('','', digits)
lines.source=lines.source.apply(lambda x: x.translate(num_digits))
lines.target= lines.target.apply(lambda x: x.translate(num_digits))
# Remove extra spaces
lines.source=lines.source.apply(lambda x: x.strip())
lines.target=lines.target.apply(lambda x: x.strip())
lines.source=lines.source.apply(lambda x: re.sub(" +", " ", x))
lines.target=lines.target.apply(lambda x: re.sub(" +", " ", x))

lines

,source,target,comments
0,general,सामान्य,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,any recent chills night sweats or fever,किसी भी हाल में ठंड लगना रात को पसीना या बुखार,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
2,have you gained or lost weight without trying,क्या आपने बिना प्रयास किए वजन घटाया या प्राप्त...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,have you been tired,क्या आप थक गए हैं,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,skin,त्वचा,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
...,...,...,...
88,any pain with sex,सेक्स के साथ कोई दर्द,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
89,any itching rash or discharge,कोई खुजली दाने या डिस्चार्ज,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
90,any lesions or sores,कोई घाव,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
91,any hernias,कोई हर्निया,CC-BY 2.0 (France) Attribution: tatoeba.org #6...


In [5]:
# Add start and end tokens to target sequences
lines.target = lines.target.apply(lambda x : 'START_ '+ x + ' _END')
lines.sample(6)

,source,target,comments
86,malefemale genitalia,START_ पुरुष महिला गुप्तांग _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
76,any muscle cramping or pain,START_ किसी भी मांसपेशियों में ऐंठन या दर्द _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
89,any itching rash or discharge,START_ कोई खुजली दाने या डिस्चार्ज _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
22,do you wear a hearing aid,START_ क्या आप हियरिंग एड पहनते हैं _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
30,any lumps or bumps in your neck,START_ आपके गले में कोई गांठ _END,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
52,any breast lumps pain or soreness,START_ कोई स्तन गांठ दर्द या कष्ट _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...


In [6]:
# Find all the source and target words and sort them
# Vocabulary of Source language
all_source_words=set()
for source in lines.source:
    for word in source.split():
        if word not in all_source_words:
            all_source_words.add(word)
# Vocabulary of Target 
all_target_words=set()
for target in lines.target:
    for word in target.split():
        if word not in all_target_words:
            all_target_words.add(word)
# sort all unique source and target words
source_words= sorted(list(all_source_words))
target_words=sorted(list(all_target_words))

In [7]:
#Find maximum sentence length in  the source and target data
source_length_list=[]
for l in lines.source:
    source_length_list.append(len(l.split(' ')))
max_source_length= max(source_length_list)
print(" Max length of the source sentence",max_source_length)
target_length_list=[]
for l in lines.target:
    target_length_list.append(len(l.split(' ')))
max_target_length= max(target_length_list)
print(" Max length of the target sentence",max_target_length)

 Max length of the source sentence 17
 Max length of the target sentence 25


In [8]:
# creating a word to index(word2idx) for source and target
source_word2idx= dict([(word, i+1) for i,word in enumerate(source_words)])
target_word2idx=dict([(word, i+1) for i, word in enumerate(target_words)])

In [9]:
#creating a dictionary for index to word for source and target vocabulary
source_idx2word= dict([(i, word) for word, i in  source_word2idx.items()])
print(source_idx2word)
target_idx2word =dict([(i, word) for word, i in target_word2idx.items()])

{1: 'a', 2: 'acheabdominal', 3: 'aid', 4: 'amounts', 5: 'and', 6: 'any', 7: 'anyone', 8: 'anywhere', 9: 'are', 10: 'areas', 11: 'arms', 12: 'at', 13: 'awake', 14: 'back', 15: 'balance', 16: 'beats', 17: 'bed', 18: 'been', 19: 'belly', 20: 'better', 21: 'black', 22: 'bleeding', 23: 'blood', 24: 'bloody', 25: 'blurred', 26: 'breast', 27: 'breasts', 28: 'breath', 29: 'breathing', 30: 'bruise', 31: 'bumps', 32: 'burning', 33: 'buttocks', 34: 'can', 35: 'cannot', 36: 'change', 37: 'changed', 38: 'changes', 39: 'chest', 40: 'chestrespiratory', 41: 'chills', 42: 'color', 43: 'congestion', 44: 'constipation', 45: 'contacts', 46: 'coordination', 47: 'cough', 48: 'cramping', 49: 'cvs', 50: 'date', 51: 'deep', 52: 'depressed', 53: 'diarrhea', 54: 'discharge', 55: 'dizziness', 56: 'do', 57: 'does', 58: 'doing', 59: 'don’t', 60: 'double', 61: 'down', 62: 'drive', 63: 'dry', 64: 'ear', 65: 'ears', 66: 'easily', 67: 'eg', 68: 'ever', 69: 'exercise', 70: 'exertion', 71: 'exposed', 72: 'eyes', 73: 'fai

In [10]:
#Shuffle the data
lines = shuffle(lines)

In [12]:
# Train - Test Split
X, y = lines.source, lines.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)
X_train.shape, X_test.shape

((88,), (5,))

In [13]:
# Input tokens for encoder
num_encoder_tokens=len(source_words)
# Input tokens for decoder zero padded
num_decoder_tokens=len(target_words) +1

In [16]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
 ''' Generate a batch of data '''
 while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_source_length),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_target_length),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_target_length, num_decoder_tokens),dtype='float32')
        for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
            for t, word in enumerate(input_text.split()):
                encoder_input_data[i, t] = source_word2idx[word] 
            for t, word in enumerate(target_text.split()):
                if t<len(target_text.split())-1:
                    decoder_input_data[i, t] = target_word2idx[word] # decoder input seq
                    if t>0:
 # decoder target sequence (one hot encoded)
 # does not include the START_ token
 # Offset by one timestep
 #print(word)
                        decoder_target_data[i, t - 1, target_word2idx[word]] = 1.
        yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [54]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 1
epochs = 10
latent_dim=358

In [55]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [56]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
 initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [57]:
# Encode the input sequence to get the "Context vectors"
encoder_model = Model(encoder_inputs, encoder_states)# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_input_h, decoder_state_input_c]# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_state_input)
decoder_states2 = [state_h2, state_c2]
# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)# Final decoder model
decoder_model = Model(
 [decoder_inputs] + decoder_state_input,
 [decoder_outputs2] + decoder_states2)

In [58]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of 
    #target sequence with the start character.
    target_seq[0, 0] = target_word2idx['START_']
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word =target_idx2word[sampled_token_index]
        decoded_sentence += ' '+ sampled_word# Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '_END' or
            len(decoded_sentence) > 50):
            stop_condition = True# Update the target sequence (of length 1).
            target_seq = np.zeros((1,1))
            target_seq[0, 0] = sampled_token_index# Update states
            states_value = [h, c]
    return decoded_sentence

In [59]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence as per data:', X_train[k:k+1].values[0])
print('Actual Hindi Translation as per data:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation predicted by model:', decoded_sentence[:-4])

Input English sentence as per data: any weakness
Actual Hindi Translation as per data:  कोई कमजोरी 
Predicted Hindi Translation predicted by model:  यह यह यह यह यह यह यह यह यह यह यह यह यह यह यह य


In [61]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
k=1
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_test[k:k+1].values[0])
print('Actual Target Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence[:-4])

Input Source sentence: any cough or phlegm production
Actual Target Translation:  कोई खांसी या कफ का उत्पादन 
Predicted Target Translation:  गैस्ट्रोइंटेस्टाइनल गैस्ट्रोइंटेस्टाइनल गैस्ट्रोइंटेस्ट
